# BICS_ABM
Ethan Roubenoff

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from BICS_ABM import BICS_ABM, VaccineRule
import time
from joblib import Parallel, delayed, Memory

/Users/eroubenoff/BICS_ABM/BICS_ABM.py:77: DtypeWarning: Columns (26,63,64,65,66,68,69,70,71,72,73,76,86,139,140,149,175,176,177,191,195,205,213,214,215,216,217,218,219,220,221,222,223,224,225,227,228,229) have mixed types. Specify dtype option on import or set low_memory=False.
  def load_data():


## Quick, demonstrative example:

In [ ]:
result = BICS_ABM( 
        N_HH = 10000,
        SEED = 4949, 
        RHO = 0.9,
        ALPHA = 0.25,
        BETA = 0.01,
        T0 = 60,
        ISOLATION_MULTIPLIER = 0.1, 
        N_VAX_DAILY = 1500,
        vax_rules = [
            VaccineRule("age > 80"),
            VaccineRule("age > 70"),
            VaccineRule("age > 60"),
            VaccineRule("age > 50", hesitancy = 0.5),
            VaccineRule("age > 40", hesitancy = 0.5),
            VaccineRule("age > 30", hesitancy = 0.5),
            VaccineRule(general = True, hesitancy = 0.5),
        ])

result.plot_trajectory()

## Another demonstrative example

Vary beta (probability of transmission from 0 to 0.1 in increments of 0.001. Run 10 simulations of each value for  beta. Plot the number of deaths against beta.

In [ ]:
beta_vec = np.arange(0, 0.1, 0.001)
beta_vec = beta_vec.repeat(10)
result_vec= np.zeros(beta_vec.shape) 

t = time.time()
for i, v in enumerate(beta_vec):
    print(i, "of", len(beta_vec), "sims", end = "\r")
    time.sleep(0) # To make carraige return work
    result = BICS_ABM(BETA = v, N_HH = 100, silent = True)
    result_vec[i] = result.D[-1] / (result.S[0] + result.E[0])
    
t = time.time() - t

print(len(beta_vec), "iterations took", t, "seconds")

In [ ]:
plt.scatter(beta_vec, result_vec*100)


# Main analysis

Run simulations that gradually decrease number of nonhousehold contacts required for priority access. Start at 30
and work down. Begin by assuming no asymptomatic transmission and 70% symptomatic rate, no isolation.


Run 10 simulations for each, and chart the number of deaths.

In [2]:
# Define a location to store cache
location = './cache'
memory = Memory(location, verbose=0)

NameError: name 'Memory' is not defined

In [ ]:
cc_min = np.arange(25, 1, -1)
cc_min = cc_min.repeat(100)

t = time.time()


def multi_fn(i):
    """ Wrapper function for multiprocessing
    
    """
    
    result = BICS_ABM(
        # 1000 HHs, approx 3000 people, and 10% chance of transmission
        N_HH = 1000, BETA = 0.01,
        # No presymptomatic transmission, all cases are symptomatic
        ALPHA = 0, RHO = 1, 
        # No isolation
        ISOLATION_MULTIPLIER = 1,
        silent = True , 
        vax_rules = [
            VaccineRule("num_cc_nonhh > " + str(i)) 
        ]
    )
    
    return result.D[-1] / (result.S[0] + result.E[0])
    
    
multi_fn = memory.cache(multi_fn)
# 10 is about the efficiency limit-- 2400 cycles will take about 7 mins
result_vec = Parallel(n_jobs=10, verbose = 50, prefer="threads")(delayed(multi_fn)(v) for v in cc_min)
    
    
t = time.time() - t

print(len(cc_min), "iterations took", t, "seconds")

plt.scatter(cc_min, result_vec)

In [ ]:
import gc; gc.collect()

In [ ]:
result.plot_trajectory()